In [1]:
import numpy as np
import pandas as pd
import time
from datetime import date

In [2]:
df_full = pd.read_parquet("content/stocks_df_combined_2025_06_13.parquet.brotli", )

In [3]:
GROWTH = [g for g in df_full.keys() if (g.find('growth_')==0)&(g.find('future')<0)]
GROWTH

['growth_1d',
 'growth_3d',
 'growth_7d',
 'growth_30d',
 'growth_90d',
 'growth_365d',
 'growth_dax_1d',
 'growth_dax_3d',
 'growth_dax_7d',
 'growth_dax_30d',
 'growth_dax_90d',
 'growth_dax_365d',
 'growth_snp500_1d',
 'growth_snp500_3d',
 'growth_snp500_7d',
 'growth_snp500_30d',
 'growth_snp500_90d',
 'growth_snp500_365d',
 'growth_dji_1d',
 'growth_dji_3d',
 'growth_dji_7d',
 'growth_dji_30d',
 'growth_dji_90d',
 'growth_dji_365d',
 'growth_epi_1d',
 'growth_epi_3d',
 'growth_epi_7d',
 'growth_epi_30d',
 'growth_epi_90d',
 'growth_epi_365d',
 'growth_gold_1d',
 'growth_gold_3d',
 'growth_gold_7d',
 'growth_gold_30d',
 'growth_gold_90d',
 'growth_gold_365d',
 'growth_wti_oil_1d',
 'growth_wti_oil_3d',
 'growth_wti_oil_7d',
 'growth_wti_oil_30d',
 'growth_wti_oil_90d',
 'growth_wti_oil_365d',
 'growth_brent_oil_1d',
 'growth_brent_oil_3d',
 'growth_brent_oil_7d',
 'growth_brent_oil_30d',
 'growth_brent_oil_90d',
 'growth_brent_oil_365d',
 'growth_btc_usd_1d',
 'growth_btc_usd_3d',


In [4]:
OHLCV = ['Open','High','Low','Close','Adj Close_x','Volume']

In [5]:
CATEGORICAL = ['Month', 'Weekday', 'Ticker', 'ticker_type']

In [6]:
TO_PREDICT = [g for g in df_full.keys() if (g.find('future')>=0)]
TO_PREDICT

['growth_future_30d', 'is_positive_growth_30d_future']

In [7]:
TO_DROP = ['Year','Date','index_x', 'index_y', 'index', 'Quarter','Adj Close_y'] + CATEGORICAL + OHLCV
TO_DROP

['Year',
 'Date',
 'index_x',
 'index_y',
 'index',
 'Quarter',
 'Adj Close_y',
 'Month',
 'Weekday',
 'Ticker',
 'ticker_type',
 'Open',
 'High',
 'Low',
 'Close',
 'Adj Close_x',
 'Volume']

In [8]:
df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))

/tmp/ipykernel_13509/912909511.py:1: RuntimeWarning: divide by zero encountered in log
  df_full['ln_volume'] = df_full.Volume.apply(lambda x: np.log(x))


In [9]:
CUSTOM_NUMERICAL = ['SMA10', 'SMA20', 'growing_moving_average', 'high_minus_low_relative','volatility', 'ln_volume']

In [10]:
TECHNICAL_INDICATORS = ['adx', 'adxr', 'apo', 'aroon_1','aroon_2', 'aroonosc',
 'bop', 'cci', 'cmo','dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
 'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
 'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
 'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
 'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod', 'ht_dcphase',
 'ht_phasor_inphase', 'ht_phasor_quadrature', 'ht_sine_sine', 'ht_sine_leadsine',
 'ht_trendmod', 'avgprice', 'medprice', 'typprice', 'wclprice']

In [11]:
TECHNICAL_PATTERNS = [g for g in df_full.keys() if g.find('cdl')>=0]
print(f'Technical patterns count = {len(TECHNICAL_PATTERNS)}, examples = {TECHNICAL_PATTERNS[0:5]}')

Technical patterns count = 61, examples = ['cdl2crows', 'cdl3blackrows', 'cdl3inside', 'cdl3linestrike', 'cdl3outside']


In [12]:
MACRO = ['gdppot_us_yoy', 'gdppot_us_qoq', 'cpi_core_yoy', 'cpi_core_mom', 'FEDFUNDS',
 'DGS1', 'DGS5', 'DGS10']

In [13]:
NUMERICAL = GROWTH + TECHNICAL_INDICATORS + TECHNICAL_PATTERNS + CUSTOM_NUMERICAL + MACRO

In [14]:
OTHER = [k for k in df_full.keys() if k not in OHLCV + CATEGORICAL + NUMERICAL + TO_DROP]
OTHER

['Close_x',
 'Dividends',
 'Stock Splits',
 'growth_future_30d',
 'is_positive_growth_30d_future',
 'Close_y']

In [15]:
df_full.Ticker.nunique()

33

In [16]:
df = df_full[df_full.Date>='2000-01-01']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Columns: 204 entries, Open to ln_volume
dtypes: datetime64[ns](3), float64(130), int32(64), int64(5), object(2)
memory usage: 253.1+ MB


In [17]:
CATEGORICAL

['Month', 'Weekday', 'Ticker', 'ticker_type']

In [18]:
df.loc[:,'Month']

3490   2000-01-01
3491   2000-01-01
3492   2000-01-01
3493   2000-01-01
3494   2000-01-01
          ...    
5696   2025-06-01
5697   2025-06-01
5698   2025-06-01
5699   2025-06-01
5700   2025-06-01
Name: Month, Length: 191795, dtype: datetime64[ns]

In [19]:
df.loc[:,'Month'] = df.Month.dt.strftime('%B')
df.loc[:,'Weekday'] = df.Weekday.astype(str)

/tmp/ipykernel_13509/2159284025.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['January' 'January' 'January' ... 'June' 'June' 'June']' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[:,'Month'] = df.Month.dt.strftime('%B')
/tmp/ipykernel_13509/2159284025.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '1' '2' ... '2' '3' '4']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[:,'Weekday'] = df.Weekday.astype(str)


### Question 1: Dummies for Month and Week-of-Month

**What is the ABSOLUTE CORRELATION VALUE of the most correlated dummy variable <month>_w<week_of_month> with the binary outcome `is_positive_growth_30d_future`?**

From the correlation analysis and modeling, you may have observed that October and November are potentially important seasonal months. In this task, you'll go further by generating dummy variables for both the **Month** and **Week-of-Month** (starting from 1). For example, the first week of October should be coded as: `'October_w1'`.

Once you've generated these new variables, identify the one with the **highest absolute correlation** with `is_positive_growth_30d_future`, and round the result to **three decimal places**.


#### Suggested Steps

1. Use [this StackOverflow reference](https://stackoverflow.com/questions/25249033/week-of-a-month-pandas) to compute the week of the month using the following formula:
  ```python
  (d.day - 1) // 7 + 1
  ```
2. Create a new string variable that combines the month name and week of the month.
Example: 'October_w1', 'November_w2', etc.

3. Add the new variable (e.g., `month_wom`) to your set of **categorical features**.

   Your updated categorical feature list should include:
   - `'Month'`
   - `'Weekday'`
   - `'Ticker'`
   - `'ticker_type'`
   - `'month_wom'`

4. Use [`pandas.get_dummies()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) to generate dummy variables for all categorical features.

   This should result in approximately **115 dummy variables**, including around **60** for the `month_wom` feature (`12 months × up to 5 weeks`).

5. Use [`DataFrame.corr()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) to compute the correlation between each feature and the target variable `is_positive_growth_30d_future`.

6. Filter the correlation results to include only the dummy variables generated from `month_wom`.

7. Create a new column named `abs_corr` in the correlation results that stores the **absolute value** of the correlations.

8. Sort the correlation results by `abs_corr` in **descending** order.

9. Identify and report the **highest absolute correlation value** among the `month_wom` dummy variables, rounded to **three decimal places**.


**NOTE**: new dummies will be used as features in the next tasks, please leave them in the dataset.

---

In [21]:
df['month_wom'] = df['Month'] + '_w' + ((df['Date'].dt.day - 1) // 7 + 1).astype(str)

/tmp/ipykernel_13509/403987377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month_wom'] = df['Month'] + '_w' + ((df['Date'].dt.day - 1) // 7 + 1).astype(str)


In [22]:
df['month_wom']

3490    January_w1
3491    January_w1
3492    January_w1
3493    January_w1
3494    January_w1
           ...    
5696       June_w2
5697       June_w2
5698       June_w2
5699       June_w2
5700       June_w2
Name: month_wom, Length: 191795, dtype: object

In [23]:
CATEGORICAL += ['month_wom']

In [24]:
CATEGORICAL

['Month', 'Weekday', 'Ticker', 'ticker_type', 'month_wom']

In [25]:
dummy_variables = pd.get_dummies(df[CATEGORICAL], dtype='int32')

In [26]:
dummy_variables.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Columns: 115 entries, Month_April to month_wom_September_w5
dtypes: int32(115)
memory usage: 85.6 MB


In [27]:
DUMMIES = dummy_variables.keys().to_list()

In [28]:
df_with_dummies = pd.concat([df, dummy_variables], axis=1)

In [29]:
df_with_dummies[NUMERICAL+DUMMIES].info()

<class 'pandas.core.frame.DataFrame'>
Index: 191795 entries, 3490 to 5700
Columns: 299 entries, growth_1d to month_wom_September_w5
dtypes: float64(121), int32(177), int64(1)
memory usage: 309.5 MB


In [30]:
corr_is_positive_growth_30d_future = df_with_dummies[NUMERICAL+DUMMIES+TO_PREDICT].corr()['is_positive_growth_30d_future']

In [31]:
corr_is_positive_growth_30d_future_df = pd.DataFrame(corr_is_positive_growth_30d_future)

In [32]:
corr_is_positive_growth_30d_future_df[corr_is_positive_growth_30d_future_df.iloc[:, 0].isin(DUMMIES)]

,is_positive_growth_30d_future


In [33]:
corr_is_positive_growth_30d_future

growth_1d                       -0.012772
growth_3d                       -0.016461
growth_7d                       -0.018023
growth_30d                      -0.023682
growth_90d                      -0.010986
                                   ...   
month_wom_September_w3          -0.006012
month_wom_September_w4           0.013558
month_wom_September_w5           0.012684
growth_future_30d                0.696468
is_positive_growth_30d_future    1.000000
Name: is_positive_growth_30d_future, Length: 301, dtype: float64

In [34]:
abs_corr = corr_is_positive_growth_30d_future_df.loc[corr_is_positive_growth_30d_future_df.index.isin('month_wom_' + df['month_wom'].values)].apply(lambda x: abs(x))

In [35]:
abs_corr['is_positive_growth_30d_future_round'] = abs_corr['is_positive_growth_30d_future'].round(3)

In [36]:
abs_corr.sort_values(by='is_positive_growth_30d_future', ascending=False).head(8)

,is_positive_growth_30d_future,is_positive_growth_30d_future_round
month_wom_October_w4,0.024968,0.025
month_wom_November_w3,0.022097,0.022
month_wom_November_w2,0.018822,0.019
month_wom_January_w2,0.018327,0.018
month_wom_October_w3,0.017734,0.018
month_wom_January_w5,0.017437,0.017
month_wom_January_w3,0.016737,0.017
month_wom_February_w1,0.016700,0.017


### Question 2:  Define New "Hand" Rules on Macro and Technical Indicator Variables 

**What is the precision score for the best of the NEW predictions (`pred3` or `pred4`), rounded to 3 digits after the comma?**

In this task, you'll apply insights from the **visualized decision tree (`clf10`)** (see *Code Snippet 5: 1.4.4 Visualisation*) to manually define and evaluate new predictive rules.


1. **Define two new 'hand' rules** based on branches that lead to 'positive' predictions in the tree:
   - `pred3_manual_dgs10_5`:  
     ```python
     (DGS10 <= 4) & (DGS5 <= 1)
     ```
   - `pred4_manual_dgs10_fedfunds`:  
     ```python
     (DGS10 > 4) & (FEDFUNDS <= 4.795)
     ```
   > **Hint**: This is not exactly the same condition as in the estimated tree (original: `(DGS10 <= 4.825) & (DGS5 <= 0.745)`; `(DGS10 > 4.825) & (FEDFUNDS <= 4.795)`), since in that case, there are no true positive predictions for both variables. Consider why this might be the case.


2. **Extend Code Snippet 3** (Manual "hand rule" predictions):  
   - Implement and apply the above two rules (`pred3`, `pred4`) to your dataset.
   - Add the resulting predictions as new columns in your dataframe (e.g., `new_df`).

3. **Compute precision**:
   - For the rule that **does** make positive predictions on the TEST set, compute its **precision score**.
   - Use standard precision metrics (`TP / (TP + FP)`).
   - Round the precision score to **three decimal places**.  
     Example: If your result is `0.57897`, your final answer should be: `0.579`.
  > **Hint**: This should already be visible in the code output, as the `IS_CORRECT` and `PREDICTIONS` sets should automatically include the new columns.



---


In [37]:
def temporal_split(df, min_date, max_date, train_prop=0.7, val_prop=0.15, test_prop=0.15):
    """
    Splits a DataFrame into three buckets based on the temporal order of the 'Date' column.

    Args:
        df (DataFrame): The DataFrame to split.
        min_date (str or Timestamp): Minimum date in the DataFrame.
        max_date (str or Timestamp): Maximum date in the DataFrame.
        train_prop (float): Proportion of data for training set (default: 0.6).
        val_prop (float): Proportion of data for validation set (default: 0.2).
        test_prop (float): Proportion of data for test set (default: 0.2).

    Returns:
        DataFrame: The input DataFrame with a new column 'split' indicating the split for each row.
    """
    # Define the date intervals
    train_end = min_date + pd.Timedelta(days=(max_date - min_date).days * train_prop)
    val_end = train_end + pd.Timedelta(days=(max_date - min_date).days * val_prop)

    # Assign split labels based on date ranges
    split_labels = []
    for date in df['Date']:
        if date <= train_end:
            split_labels.append('train')
        elif date <= val_end:
            split_labels.append('validation')
        else:
            split_labels.append('test')

    # Add 'split' column to the DataFrame
    df['split'] = split_labels

    return df

In [38]:
min_date_df = df_with_dummies.Date.min()
max_date_df = df_with_dummies.Date.max()

df_with_dummies = temporal_split(df_with_dummies,
                                 min_date = min_date_df,
                                 max_date = max_date_df)

In [39]:
df_with_dummies['split'].value_counts()/len(df_with_dummies)

split
train         0.676399
test          0.163758
validation    0.159843
Name: count, dtype: float64

In [40]:
new_df = df_with_dummies.copy()

In [41]:
new_df['pred0_manual_cci'] = (new_df.cci>200).astype(int)
new_df['pred1_manual_prev_g1'] = (new_df.growth_30d>1).astype(int)
new_df['pred2_manual_prev_g1_and_snp'] = ((new_df['growth_30d'] > 1) & (new_df['growth_snp500_30d'] > 1)).astype(int)

In [42]:
new_df['pred3_manual_dgs10_5'] = ((new_df['DGS10'] <= 4) & (new_df['DGS5'] <= 1)).astype(int)

In [43]:
new_df['pred4_manual_dgs10_fedfunds'] = ((new_df['DGS10'] > 4) & (new_df['FEDFUNDS'] <= 4.795)).astype(int)

In [44]:
PREDICTIONS = [k for k in new_df.keys() if k.startswith('pred')]
PREDICTIONS

['pred0_manual_cci',
 'pred1_manual_prev_g1',
 'pred2_manual_prev_g1_and_snp',
 'pred3_manual_dgs10_5',
 'pred4_manual_dgs10_fedfunds']

In [45]:
for pred in PREDICTIONS:
  part1 = pred.split('_')[0] # first prefix before '_'
  new_df[f'is_correct_{part1}'] =  (new_df[pred] == new_df.is_positive_growth_30d_future).astype(int)

In [46]:
IS_CORRECT =  [k for k in new_df.keys() if k.startswith('is_correct_')]
IS_CORRECT

['is_correct_pred0',
 'is_correct_pred1',
 'is_correct_pred2',
 'is_correct_pred3',
 'is_correct_pred4']

In [47]:
new_df[PREDICTIONS+IS_CORRECT+['is_positive_growth_30d_future']].sample(5)

,pred0_manual_cci,pred1_manual_prev_g1,pred2_manual_prev_g1_and_snp,pred3_manual_dgs10_5,pred4_manual_dgs10_fedfunds,is_correct_pred0,is_correct_pred1,is_correct_pred2,is_correct_pred3,is_correct_pred4,is_positive_growth_30d_future
5707,0,0,0,0,1,0,0,0,0,1,1
8253,0,1,0,0,0,0,1,0,0,0,1
1703,0,0,0,0,0,0,0,0,0,0,1
2330,0,0,0,0,0,1,1,1,1,1,0
1960,0,0,0,0,0,0,0,0,0,0,1


In [48]:
len(new_df[new_df.split=='test'])

31408

In [49]:
# define "Precision" for ALL predictions on a Test dataset (~4 last years of trading)
for i,column in enumerate(IS_CORRECT):
  prediction_column = PREDICTIONS[i]
  is_correct_column = column
  filter = (new_df.split=='test') & (new_df[prediction_column]==1)
  print(f'Prediction column:{prediction_column} , is_correct_column: {is_correct_column}')
  print(new_df[filter][is_correct_column].value_counts())
  print((new_df[filter][is_correct_column].value_counts()/len(new_df[filter])).round(3))

  print('---------')

Prediction column:pred0_manual_cci , is_correct_column: is_correct_pred0
is_correct_pred0
1    443
0    351
Name: count, dtype: int64
is_correct_pred0
1    0.558
0    0.442
Name: count, dtype: float64
---------
Prediction column:pred1_manual_prev_g1 , is_correct_column: is_correct_pred1
is_correct_pred1
1    9748
0    8243
Name: count, dtype: int64
is_correct_pred1
1    0.542
0    0.458
Name: count, dtype: float64
---------
Prediction column:pred2_manual_prev_g1_and_snp , is_correct_column: is_correct_pred2
is_correct_pred2
1    6984
0    6383
Name: count, dtype: int64
is_correct_pred2
1    0.522
0    0.478
Name: count, dtype: float64
---------
Prediction column:pred3_manual_dgs10_5 , is_correct_column: is_correct_pred3
is_correct_pred3
1    578
0    419
Name: count, dtype: int64
is_correct_pred3
1    0.58
0    0.42
Name: count, dtype: float64
---------
Prediction column:pred4_manual_dgs10_fedfunds , is_correct_column: is_correct_pred4
is_correct_pred4
0    3020
1    2640
Name: count, 

### Question 3: Unique Correct Predictions from a 10-Level Decision Tree Classifier (`pred5_clf_10`)

**What is the total number of records in the TEST dataset where the new prediction `pred5_clf_10` is correct, while all 'hand' rule predictions (`pred0` to `pred4`) are incorrect?**


To ensure reproducibility, please include the following parameter in the Decision Tree Classifier:
```python
clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42) 
```


#### Step 1: Train the Decision Tree and Generate Predictions

- Initialize a Decision Tree Classifier with a maximum depth of 10 and set `random_state=42` for reproducibility.
- Fit the classifier on the combined TRAIN and VALIDATION datasets.
- Use the trained model to predict on the entire dataset (TRAIN + VALIDATION + TEST).
- Store these predictions in a new column named `pred5_clf_10` within your main dataframe.
- **Hint**: When predicting on the entire dataset, it's easy to join the predictions with the full DataFrame, since the number of records and their order remain the same. You will need to define X_all and y_all and apply the same cleaning steps used previously for X_train, y_train, X_test, and y_test. This makes it straightforward to define a new column, for example:  
  ```python
  df['pred5_clf_10'] = <predictions vector from clf10.predict(X_all)>

#### Step 2: Identify Unique Correct Predictions by `pred5_clf_10`

- Create a new boolean column, `only_pred5_is_correct`, that is `True` only when:
  - The prediction from `pred5_clf_10` is correct (i.e., matches the true label).
  - All other hand rule predictions (`pred0` through `pred4`) are incorrect.

#### Step 3: Count Unique Correct Predictions on the TEST Set

- Convert the `only_pred5_is_correct` column from boolean to integer.
- Filter the dataframe for records belonging to the TEST dataset.
- Count how many records in the TEST set have `only_pred5_is_correct` equal to 1.
- Report this count as your final answer.

### Advanced (Optional)

- To generalize this for many prediction columns (e.g., `pred0` to `pred99`), define a function that can be applied to an entire dataframe row.
- This function should identify whether a specific prediction (`predX`) is uniquely correct (correct while all others are incorrect).
- This approach avoids hardcoding conditions for each predictor and scales easily.
- For examples of how to apply functions to rows in pandas, see this helpful resource:  
  [Pandas apply function to every row](https://sparkbyexamples.com/pandas/pandas-apply-function-to-every-row/)

---


In [50]:
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [51]:
features_list = NUMERICAL+DUMMIES
to_predict = 'is_positive_growth_30d_future'

train_df = new_df[new_df.split.isin(['train','validation'])].copy(deep=True)
test_df = new_df[new_df.split.isin(['test'])].copy(deep=True)

# ONLY numerical Separate features and target variable for training and testing sets
# need Date and Ticker later when merging predictions to the dataset
X_train = train_df[features_list+[to_predict,'Date','Ticker']]
X_test = test_df[features_list+[to_predict,'Date','Ticker']]

print(f'length: X_train {X_train.shape},  X_test {X_test.shape}')


length: X_train (160387, 302),  X_test (31408, 302)


In [52]:
X_full = new_df[features_list+[to_predict,'Date','Ticker']].copy(deep=True)

In [53]:
pd.options.mode.chained_assignment = None  # default='warn'

X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Need to fill NaNs somehow
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

print(f'length: X_train_imputed {X_train.shape},  X_test_imputed {X_test.shape}')

length: X_train_imputed (160387, 302),  X_test_imputed (31408, 302)


In [54]:
X_full.replace([np.inf, -np.inf], np.nan, inplace=True)
X_full.fillna(0, inplace=True)
print(f'length: X_full {X_full.shape}')

length: X_full (191795, 302)


In [55]:
X_train_imputed = X_train
X_test_imputed = X_test

In [56]:
print(f'length: X_train_imputed {X_train_imputed.shape},  X_test_imputed {X_test_imputed.shape}')

length: X_train_imputed (160387, 302),  X_test_imputed (31408, 302)


In [57]:
y_train = X_train_imputed[to_predict]
y_test = X_test_imputed[to_predict]

# remove y_train, y_test from X_ dataframes
del X_train_imputed[to_predict]
del X_test_imputed[to_predict]

In [58]:
y_full = X_full[to_predict]
del X_full[to_predict]

In [59]:
def fit_decision_tree(X, y, max_depth):
  clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42)
  clf.fit(X, y)
  return clf, X.columns

In [60]:
%%time

clf_10, train_columns = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1),
                           y=y_train,
                           max_depth=10)

CPU times: user 17.4 s, sys: 508 ms, total: 17.9 s
Wall time: 17.9 s


In [61]:
def predict_decision_tree(clf:DecisionTreeClassifier, df_X:pd.DataFrame, y_true: pd.Series):
  # Predict the target variable on the test data
  y_pred = clf.predict(df_X)

  max_depth = clf.tree_.max_depth
  # Print the maximum depth
  print("Maximum depth of the decision tree:", max_depth)

  # Calculate the accuracy/precision of the model
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  print(f'Accuracy ={accuracy}, precision = {precision}')

  # resulting df
  result_df = pd.concat([df_X, y_true, pd.Series(y_pred, index=df_X.index, name='pred_')], axis=1)

  return result_df, accuracy, precision

In [62]:
pred10, _, _ = predict_decision_tree(clf_10, X_full.drop(['Date','Ticker'],axis=1), y_full)

Maximum depth of the decision tree: 10
Accuracy =0.6958314867436586, precision = 0.6950666328571831


In [63]:
pred10.pred_.value_counts()

pred_
1    141972
0     49823
Name: count, dtype: int64

In [68]:
pred10_test, _, _ = predict_decision_tree(clf_10, X_test_imputed.drop(['Date','Ticker'],axis=1), y_test)

Maximum depth of the decision tree: 10
Accuracy =0.5571510443199185, precision = 0.5889278426037361


In [69]:
pred10_test.pred_.value_counts()

pred_
1    19111
0    12297
Name: count, dtype: int64

In [83]:
test_df['pred5_clf_10'] = pred10_test['pred_']

In [88]:
test_df['only_pred5_is_correct'] = (test_df['pred5_clf_10'] == test_df[to_predict] & (test_df[IS_CORRECT] == 0).all(axis=1)).astype(int)

In [87]:
IS_CORRECT

['is_correct_pred0',
 'is_correct_pred1',
 'is_correct_pred2',
 'is_correct_pred3',
 'is_correct_pred4']

In [90]:
test_df['only_pred5_is_correct'].value_counts()

only_pred5_is_correct
0    17459
1    13949
Name: count, dtype: int64

In [91]:
new_df['pred5_clf_10'] = pred10['pred_']

In [92]:
PREDICTIONS

['pred0_manual_cci',
 'pred1_manual_prev_g1',
 'pred2_manual_prev_g1_and_snp',
 'pred3_manual_dgs10_5',
 'pred4_manual_dgs10_fedfunds']

In [120]:
new_df['only_pred5_is_correct'] = ((new_df['pred5_clf_10'] == new_df['is_positive_growth_30d_future']) & (new_df[IS_CORRECT] == 0).all(axis=1)).astype(int)

In [121]:
new_df["only_pred5_is_correct"].value_counts()

only_pred5_is_correct
0    165145
1     26650
Name: count, dtype: int64

In [116]:
to_predict

'is_positive_growth_30d_future'

In [117]:
new_df[PREDICTIONS + ['pred5_clf_10', 'only_pred5_is_correct']] 

,pred0_manual_cci,pred1_manual_prev_g1,pred2_manual_prev_g1_and_snp,pred3_manual_dgs10_5,pred4_manual_dgs10_fedfunds,pred5_clf_10,only_pred5_is_correct
3490,0,1,1,0,0,0,1
3491,0,1,0,0,0,0,1
3492,0,1,0,0,0,0,1
3493,0,1,0,0,0,0,1
3494,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...
5696,0,1,1,0,1,0,1
5697,0,1,1,0,1,1,0
5698,0,1,1,0,1,1,0
5699,0,1,1,0,1,1,0


In [122]:
new_df['only_pred5_is_correct'] = ((new_df.pred5_clf_10 == new_df.is_positive_growth_30d_future) & (new_df[IS_CORRECT] == 0).all(axis=1)).astype(int)

In [123]:
new_df["only_pred5_is_correct"].value_counts()

only_pred5_is_correct
0    165145
1     26650
Name: count, dtype: int64

In [126]:
new_test_df = new_df[new_df.split.isin(['test'])]

In [127]:
new_test_df['only_pred5_is_correct'].value_counts()

only_pred5_is_correct
0    27638
1     3770
Name: count, dtype: int64

In [78]:
new_test_df.shape

(31408, 333)

In [132]:
new_test_df['only_pred5_is_correct'] = new_test_df.apply(
    lambda row: (all(row[col] == 0 for col in IS_CORRECT) & (row['pred5_clf_10']==row[to_predict])),
    axis=1
)


In [133]:
new_test_df['only_pred5_is_correct'].value_counts()

only_pred5_is_correct
False    27638
True      3770
Name: count, dtype: int64

### Question 3: Unique Correct Predictions from a 10-Level Decision Tree Classifier (`pred5_clf_10`)

**What is the total number of records in the TEST dataset where the new prediction `pred5_clf_10` is correct, while all 'hand' rule predictions (`pred0` to `pred4`) are incorrect?**


To ensure reproducibility, please include the following parameter in the Decision Tree Classifier:
```python
clf = DecisionTreeClassifier(max_depth=max_depth, random_state=42) 
```


#### Step 1: Train the Decision Tree and Generate Predictions

- Initialize a Decision Tree Classifier with a maximum depth of 10 and set `random_state=42` for reproducibility.
- Fit the classifier on the combined TRAIN and VALIDATION datasets.
- Use the trained model to predict on the entire dataset (TRAIN + VALIDATION + TEST).
- Store these predictions in a new column named `pred5_clf_10` within your main dataframe.
- **Hint**: When predicting on the entire dataset, it's easy to join the predictions with the full DataFrame, since the number of records and their order remain the same. You will need to define X_all and y_all and apply the same cleaning steps used previously for X_train, y_train, X_test, and y_test. This makes it straightforward to define a new column, for example:  
  ```python
  df['pred5_clf_10'] = <predictions vector from clf10.predict(X_all)>

#### Step 2: Identify Unique Correct Predictions by `pred5_clf_10`

- Create a new boolean column, `only_pred5_is_correct`, that is `True` only when:
  - The prediction from `pred5_clf_10` is correct (i.e., matches the true label).
  - All other hand rule predictions (`pred0` through `pred4`) are incorrect.

#### Step 3: Count Unique Correct Predictions on the TEST Set

- Convert the `only_pred5_is_correct` column from boolean to integer.
- Filter the dataframe for records belonging to the TEST dataset.
- Count how many records in the TEST set have `only_pred5_is_correct` equal to 1.
- Report this count as your final answer.

### Advanced (Optional)

- To generalize this for many prediction columns (e.g., `pred0` to `pred99`), define a function that can be applied to an entire dataframe row.
- This function should identify whether a specific prediction (`predX`) is uniquely correct (correct while all others are incorrect).
- This approach avoids hardcoding conditions for each predictor and scales easily.
- For examples of how to apply functions to rows in pandas, see this helpful resource:  
  [Pandas apply function to every row](https://sparkbyexamples.com/pandas/pandas-apply-function-to-every-row/)

---

In [201]:
%%time

accuracies = []
precisions = []
for i in range(1, 21):
    clf, _ = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1), y=y_train, max_depth=i)
    pred, accuracy, precision = predict_decision_tree(clf, X_test_imputed.drop(['Date','Ticker'],axis=1), y_test)
    accuracies.append([i, accuracy])
    precisions.append([i, precision])


Maximum depth of the decision tree: 1
Accuracy =0.5410404992358635, precision = 0.5465816507261778
Maximum depth of the decision tree: 2
Accuracy =0.5510697911360163, precision = 0.5510697911360163
Maximum depth of the decision tree: 3
Accuracy =0.5510697911360163, precision = 0.5510697911360163
Maximum depth of the decision tree: 4
Accuracy =0.5510697911360163, precision = 0.5510697911360163
Maximum depth of the decision tree: 5
Accuracy =0.5991467142129394, precision = 0.627845220030349
Maximum depth of the decision tree: 6
Accuracy =0.5498599083036169, precision = 0.5691415110800907
Maximum depth of the decision tree: 7
Accuracy =0.5701732042791645, precision = 0.5938485804416404
Maximum depth of the decision tree: 8
Accuracy =0.5762226184411615, precision = 0.5897136702270891
Maximum depth of the decision tree: 9
Accuracy =0.5642193071828834, precision = 0.5915457349446327
Maximum depth of the decision tree: 10
Accuracy =0.5571510443199185, precision = 0.5889278426037361
Maximum de

In [202]:
%%time

accuracies_full = []
precisions_full = []
for i in range(1, 21):
    clf, _ = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1), y=y_train, max_depth=i)
    pred, accuracy, precision = predict_decision_tree(clf, X_full.drop(['Date','Ticker'],axis=1), y_full)
    accuracies_full.append([i, accuracy])
    precisions_full.append([i, precision])

Maximum depth of the decision tree: 1
Accuracy =0.5981490654083788, precision = 0.6052636113520313
Maximum depth of the decision tree: 2
Accuracy =0.603316040564144, precision = 0.6015046109043845
Maximum depth of the decision tree: 3
Accuracy =0.6083943794155218, precision = 0.6059595017285847
Maximum depth of the decision tree: 4
Accuracy =0.6102140306055945, precision = 0.6073287758013782
Maximum depth of the decision tree: 5
Accuracy =0.63493313172919, precision = 0.6501869261641652
Maximum depth of the decision tree: 6
Accuracy =0.6358038530722907, precision = 0.6433776673964903
Maximum depth of the decision tree: 7
Accuracy =0.652634323105399, precision = 0.6518481743918401
Maximum depth of the decision tree: 8
Accuracy =0.6677859172554029, precision = 0.6705546119898776
Maximum depth of the decision tree: 9
Accuracy =0.6782085038713209, precision = 0.6783441237792165
Maximum depth of the decision tree: 10
Accuracy =0.6958314867436586, precision = 0.6950666328571831
Maximum depth

In [187]:
sorted_precisions = sorted(precisions, key=lambda x: x[1], reverse=True)
sorted_precisions

[[5, 0.627845220030349],
 [11, 0.5947726306842329],
 [7, 0.5938485804416404],
 [9, 0.5915457349446327],
 [16, 0.5904395868364161],
 [8, 0.5897136702270891],
 [10, 0.5889278426037361],
 [17, 0.5885812962063569],
 [19, 0.5856247284464031],
 [15, 0.5854521696520935],
 [18, 0.585290999386195],
 [13, 0.5828431861230187],
 [20, 0.5811975315505643],
 [12, 0.5788757932910245],
 [14, 0.57773410527434],
 [6, 0.5691415110800907],
 [2, 0.5510697911360163],
 [3, 0.5510697911360163],
 [4, 0.5510697911360163],
 [1, 0.5465816507261778]]

In [203]:
sorted(precisions_full, key=lambda x: x[1], reverse=True)

[[20, 0.8518668328641831],
 [19, 0.8433571810644558],
 [18, 0.8274811792425723],
 [17, 0.8091023614465566],
 [16, 0.7974463755007107],
 [15, 0.7691025348253026],
 [14, 0.7576379544953514],
 [13, 0.739768151851688],
 [12, 0.7337203912121649],
 [11, 0.7099412751677853],
 [10, 0.6950666328571831],
 [9, 0.6783441237792165],
 [8, 0.6705546119898776],
 [7, 0.6518481743918401],
 [5, 0.6501869261641652],
 [6, 0.6433776673964903],
 [4, 0.6073287758013782],
 [3, 0.6059595017285847],
 [1, 0.6052636113520313],
 [2, 0.6015046109043845]]

In [199]:
clf_5, _ = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1), y=y_train, max_depth=5)

In [200]:
pred5, _, _ = predict_decision_tree(clf_5, X_full.drop(['Date','Ticker'],axis=1), y_full)

Maximum depth of the decision tree: 5
Accuracy =0.63493313172919, precision = 0.6501869261641652


In [190]:
new_df['pred6_clf_best'] = pred5['pred_']

In [193]:
clf_20, _ = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1),
                           y=y_train,
                           max_depth=20)
pred20, _, _ = predict_decision_tree(clf_20, X_full.drop(['Date','Ticker'],axis=1), y_full)
new_df['pred7_clf_best'] = pred20['pred_']


Maximum depth of the decision tree: 20
Accuracy =0.8515341901509424, precision = 0.8518668328641831


In [196]:
new_df['pred7_clf_20'] = pred20['pred_']

In [194]:
clf_15, _ = fit_decision_tree(X=X_train_imputed.drop(['Date','Ticker'],axis=1),
                           y=y_train,
                           max_depth=15)
pred15, _, _ = predict_decision_tree(clf_15, X_full.drop(['Date','Ticker'],axis=1), y_full)
new_df['pred8_clf_best'] = pred15['pred_']


Maximum depth of the decision tree: 15
Accuracy =0.7756875830965354, precision = 0.7691025348253026


In [197]:
new_df['pred8_clf_15'] = pred15['pred_']

In [191]:
def show_precisions_pred():
    predictions = [k for k in new_df.keys() if k.startswith('pred')]
    
    for pred in predictions:
      part1 = pred.split('_')[0]
      new_df[f'is_correct_{part1}'] =  (new_df[pred] == new_df.is_positive_growth_30d_future).astype(int)

    is_correct =  [k for k in new_df.keys() if k.startswith('is_correct_')]
    
    for i,column in enumerate(is_correct):
      prediction_column = predictions[i]
      is_correct_column = column
      filter = (new_df[prediction_column]==1)
      print(f'Prediction column:{prediction_column} , is_correct_column: {is_correct_column}')
      print(new_df[filter][is_correct_column].value_counts())
      print((new_df[filter][is_correct_column].value_counts()/len(new_df[filter])).round(3))
    
      print('---------')

In [198]:
show_precisions_pred()

Prediction column:pred3_manual_dgs10_5 , is_correct_column: is_correct_pred3
is_correct_pred3
1    17761
0     8477
Name: count, dtype: int64
is_correct_pred3
1    0.677
0    0.323
Name: count, dtype: float64
---------
Prediction column:pred4_manual_dgs10_fedfunds , is_correct_column: is_correct_pred4
is_correct_pred4
1    19545
0    16726
Name: count, dtype: int64
is_correct_pred4
1    0.539
0    0.461
Name: count, dtype: float64
---------
Prediction column:pred0_manual_cci , is_correct_column: is_correct_pred0
is_correct_pred0
1    2986
0    2122
Name: count, dtype: int64
is_correct_pred0
1    0.585
0    0.415
Name: count, dtype: float64
---------
Prediction column:pred1_manual_prev_g1 , is_correct_column: is_correct_pred1
is_correct_pred1
1    66488
0    47596
Name: count, dtype: int64
is_correct_pred1
1    0.583
0    0.417
Name: count, dtype: float64
---------
Prediction column:pred2_manual_prev_g1_and_snp , is_correct_column: is_correct_pred2
is_correct_pred2
1    50829
0    36637